# Carga de datos

In [ ]:
# Cargamos las librerías que consideramos necesarias para el desarrollo del proyecto

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import time

In [ ]:
# Nos conectamos a la data que se encuentra en Google Drive

url='https://drive.google.com/file/d/1IOZDGxoWSMAlv_p0MIUkF1ZJpPrUwBK1/view?usp=sharing'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
data = pd.read_csv(url)

In [ ]:
data

Identificamos un tamaño de dataset de 1372 filas o muestras y 98 columnas, adicional se observan que las primeras 5 columnas son de referencia en la fecha, el resto de variables predictoras o dependientes hacen referencia a la utilización de los recursos productivos en kilogramos (KG) y horas (Hrs) y al final del data set identificamos los consumos diarios en energía y en agua que serían nuestas variables a predecir, las variables respuesta, se identifica una historia desde el 12/02/2018 hasta el 14/11/2021 detalle día.

# Análisis Exploratorio de los datos

Observemos la info y naturaleza de cada variable y adicionalmente si hay presencia de datos nulos

In [ ]:
data.columns

In [ ]:
data.info()

Luego de analizar el output anterior, se puede afirmar que no se encuentran datos nulos en el dataset, por lo cual posteriormente se podrá obviar la fase de imputación, también se identican que la mayoría de variables de fecha se encuentran en tipo entero, la mayoria de variables se tienen en tipo entero o flotante, lo cual esta bien dado el detalle de la medición en el reporte de los datos

Se describira el conjunto de datos en algunos estadisticos generales

In [ ]:
pd.set_option('display.max_columns', 500) # Para modificar el maximo de columnas en un output de pandas
data.describe()

Observando detenidamente el output de "describe" anterior, se afirma que el valor minimo en la mayoría de las variables numericas es 0, esto debido a que se esta contemplando los días domingos, que son los días que no hay producción. Adicionalmente se observa que algunas variables el minimo hasta el quantil 25% o incluso hasta el 50% se mantiene en 0, esto posiblemente indica un sesgo de datos causando por una subutilización del recurso o por activación solo en materiales extraordinarios o temporada

In [ ]:
sns.barplot(x=data.DiasSem, y=data.energia)

En la grafica anterior podemos observar que los dias 7 o domingos son los días de menor demanda energetica en promedio en la planta, ya que no es día de producción

Ahora se graficarán algunas frecuencias de las variables referentes a fechas, para mirar un poco la homogeneidad en los tiempos y temporadas del año

In [ ]:
plt.rcParams['figure.figsize']=(15,5)
sns.barplot(x=data.Semana.value_counts().index, y=data.Semana.value_counts(), color="royalblue")

A partir de la grafica anterior, se puede afirmar que las semanas del primer mes enero y de los 2 ultimos meses noviembre y diciembre se tienen menos observaciones

In [ ]:
sns.barplot(x=data.DiaMes.value_counts().index, y=data.DiaMes.value_counts(), color="lightgreen")

En la grafica anterior observamos que los ultimos días del mes tienen menos observaciones, debido a que no todos los meses tienen 30 o 31 días

In [ ]:
sns.barplot(x=data.Mes.value_counts().index, y=data.Mes.value_counts(), color="coral")

Al igual que lo observamos en la semana, en la variable mes se observan menos observaciones en los meses Enero, febrero, noviembre y diciembre con respecto a los demas meses, esto debido a la disponibilidad de la fuente de información

In [ ]:
sns.barplot(x=data.DiasSem.value_counts().index, y=data.DiasSem.value_counts(), color="lightgrey")

La frecuecia de datos de acuerdo al dia de la semana es bastante homogéneo

In [ ]:
data.hist(figsize=(80,80)) 

De acuerdo con lo anterior, se analizará un histograma por grupo de histogramas similares, esto con el fin del mejor entendimiento de la data y de la necesidad

In [ ]:
data.Semana.hist()

Por ejemplo en los histogramas de las variables temporales o de fechas, se observa un comportamiento de frecuencias homogeneo y paralelo

In [ ]:
data.KG_15EMPA09.hist()

El histograma del recurso 15EMPA09, es representante de una serie de recursos e histogramas que son de uso cómun en la planta, de forma permanente y en la linea de productos de una cantidad de productos considerables, por ello, a pesar de los valores alrededores de 0 que representan los días domingos, vemos un comportamiento aproximado a una normal, posiblemente con una media y mediana similares. Los recursos que tambien ofrecen el mismo comportamiento, serían: 15EMPA10, 15PREM01, 15MOLI07, 15MEZC08, 15FORM02, 15SEPA02, 15MEZC06, 15AHUM01, 15FORM01, 15SEPA01, 15EMPA11, 15EMPM01, 15MEZC05, 15MOLI03, 15EMPA04, 15TAJA02, 15EMUL01

In [ ]:
data.KG_15AHUM09.hist()

El recurso 15AHUM09, representa otro tipo de comportamiento, en el cual, excluyendo los valores proximos a 0, su distribución parece ser mas versatil, en la cual en algunos casos se prescindiría de su uso, por la priorización de otras lineas de producción dentro de la planta o incluso que con la ocupación dada en otros recursos similares como el 15AHUM01, sean suficiente en algunas temporadas del año, este comportamiento es similar en los siguientes recursos: 15EMBU01, 15EMPM06, 15TUNE01, 15MEZC07, 15MOLI01, 15MOLI06

In [ ]:
data.KG_15HORN01.hist()

El grafico del recurso 15HORN01, nos da una señal respecto a los valores de la data y posible escalas muy diferentes, esto ocurre tambien con los recursos: 15PORC03, 15TUNE03

In [ ]:
data.KG_15EMPA06.hist()

El comportamiento de la 15EMPA06, es tipico en la utilización esporadica por producción de lineas especiales o tempordas, los recursos que comparten esta distribución similar son: 15SELL02, 15EMBU03, 15EMPA08, 15EMPM05, 15COCI01, 15SELL05, 15DOSI01, 15SELL06, 15SELL03

Dada la cantidad de muestras y variables del dataset, se toma la decisión de realizar parte del análisis exploratorio con una muestra de variables, que representa alrededor del 30% del total

In [ ]:
sub_data = data[['KG_15EMPA09',
       'KG_15EMPA10', 'KG_15PREM01', 'KG_15MEZC08', 'KG_15MOLI07',
       'KG_15AHUM10', 'KG_15FORM02', 'KG_15SEPA02', 'KG_15MEZC06',
       'KG_15AHUM01', 'KG_15FORM01', 'KG_15SEPA01', 'KG_15AHUM09',
       'KG_15EMBU01', 'KG_15EMPM06', 'KG_15TUNE01', 'KG_15MEZC07',
       'Hrs_15EMPA09', 'Hrs_15EMPA10', 'Hrs_15PREM01',
       'Hrs_15MEZC08', 'Hrs_15MOLI07', 'Hrs_15AHUM10', 'Hrs_15FORM02',
       'Hrs_15SEPA02', 'Hrs_15MEZC06', 'Hrs_15AHUM01', 'Hrs_15FORM01',
       'Hrs_15SEPA01', 'Hrs_15AHUM09', 'Hrs_15EMBU01', 'Hrs_15EMPM06',
       'Hrs_15TUNE01', 'Hrs_15MEZC07','energia']]

Para las mismas variables en kg, se seleciona la información en Horas

Ahora realizaremos un analisis de dispersión de la submuestra

In [ ]:
start_time = time.time()

pd.plotting.scatter_matrix(sub_data, figsize=(100,100), grid=True, diagonal="kde")

elapsed_time = time.time() - start_time
print(elapsed_time)

A partir del output anterior, observamos la relación entre algunas variables predictoras con una relación al parecer muy fuerte y positiva, que podría significar colinealidad, tambien se identifica en algunas relaciones algun tipo de clusters, la relación con las variables respuesta no es completamente clara, en algunos casos se observa un tipo de centroide con rango de confiabilidad de variación.

A continuación, se creará un sub dataset de la data original que solo incluye las variables númericas, para analizarlas entre ellas

In [ ]:
data_numbers = data[['KG_15EMPA09',
       'KG_15EMPA10', 'KG_15PREM01', 'KG_15MEZC08', 'KG_15MOLI07',
       'KG_15AHUM10', 'KG_15FORM02', 'KG_15SEPA02', 'KG_15MEZC06',
       'KG_15AHUM01', 'KG_15FORM01', 'KG_15SEPA01', 'KG_15AHUM09',
       'KG_15EMBU01', 'KG_15EMPM06', 'KG_15TUNE01', 'KG_15MEZC07',
       'KG_15MOLI01', 'KG_15MOLI06', 'KG_15EMPA11', 'KG_15EMPM01',
       'KG_15MEZC05', 'KG_15MOLI03', 'KG_15HORN01', 'KG_15PORC03',
       'KG_15TUNE03', 'KG_15EMPA06', 'KG_15EMPA02', 'KG_15MOLI02',
       'KG_15SELL02', 'KG_15MEZC04', 'KG_15MOLI08', 'KG_15TUNE02',
       'KG_15PORC01', 'KG_15TUNE04', 'KG_15EMBU03', 'KG_15EMPA08',
       'KG_15EMPM05', 'KG_15COCI01', 'KG_15EMPA04', 'KG_15TAJA02',
       'KG_15EMUL01', 'KG_15SELL05', 'KG_15DOSI01', 'KG_15SELL06',
       'KG_15SELL03', 'Hrs_15EMPA09', 'Hrs_15EMPA10', 'Hrs_15PREM01',
       'Hrs_15MEZC08', 'Hrs_15MOLI07', 'Hrs_15AHUM10', 'Hrs_15FORM02',
       'Hrs_15SEPA02', 'Hrs_15MEZC06', 'Hrs_15AHUM01', 'Hrs_15FORM01',
       'Hrs_15SEPA01', 'Hrs_15AHUM09', 'Hrs_15EMBU01', 'Hrs_15EMPM06',
       'Hrs_15TUNE01', 'Hrs_15MEZC07', 'Hrs_15MOLI01', 'Hrs_15MOLI06',
       'Hrs_15EMPA11', 'Hrs_15EMPM01', 'Hrs_15MEZC05', 'Hrs_15MOLI03',
       'Hrs_15HORN01', 'Hrs_15PORC03', 'Hrs_15TUNE03', 'Hrs_15EMPA06',
       'Hrs_15EMPA02', 'Hrs_15MOLI02', 'Hrs_15SELL02', 'Hrs_15MEZC04',
       'Hrs_15MOLI08', 'Hrs_15TUNE02', 'Hrs_15PORC01', 'Hrs_15TUNE04',
       'Hrs_15EMBU03', 'Hrs_15EMPA08', 'Hrs_15EMPM05', 'Hrs_15COCI01',
       'Hrs_15EMPA04', 'Hrs_15TAJA02', 'Hrs_15EMUL01', 'Hrs_15SELL05',
       'Hrs_15DOSI01', 'Hrs_15SELL06', 'Hrs_15SELL03','energia']]

Desarrollamos la matrix de correlación de pearson con el fin de analizar relaciones lineales entre las variables predictoras y entre estas y las variables a predecir

In [ ]:
start_time = time.time()
Matriz_Correlacion = pd.DataFrame(np.corrcoef(data_numbers.values, rowvar=False), columns=data_numbers.columns, index=data_numbers.columns)
plt.figure(figsize=(60,60))
ax = sns.heatmap(Matriz_Correlacion, annot=True)
plt.show()
elapsed_time = time.time() - start_time
print(elapsed_time)

Como se puede identificar en la matriz del output anterior, se observa presencia de colinealidad positiva entre las variables predictoras en algunos casos posteriormente se analizará la opción de la reducción de la dimensión dado esto, con respecto a la variable respuesta tambien se observan relaciones la mayoria entre moderadas-fuertes y positivas, esto indica un buen indicio en la calidad de los datos y su potencial predictor respecto a la solución que se pretende a generar

In [ ]:
data_numbers_n = data_numbers.copy()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

min_max_scaler= MinMaxScaler()
min_max_scaler.fit(data_numbers_n[['KG_15EMPA09',
       'KG_15EMPA10', 'KG_15PREM01', 'KG_15MEZC08', 'KG_15MOLI07',
       'KG_15AHUM10', 'KG_15FORM02', 'KG_15SEPA02', 'KG_15MEZC06',
       'KG_15AHUM01', 'KG_15FORM01', 'KG_15SEPA01', 'KG_15AHUM09',
       'KG_15EMBU01', 'KG_15EMPM06', 'KG_15TUNE01', 'KG_15MEZC07',
       'KG_15MOLI01', 'KG_15MOLI06', 'KG_15EMPA11', 'KG_15EMPM01',
       'KG_15MEZC05', 'KG_15MOLI03', 'KG_15HORN01', 'KG_15PORC03',
       'KG_15TUNE03', 'KG_15EMPA06', 'KG_15EMPA02', 'KG_15MOLI02',
       'KG_15SELL02', 'KG_15MEZC04', 'KG_15MOLI08', 'KG_15TUNE02',
       'KG_15PORC01', 'KG_15TUNE04', 'KG_15EMBU03', 'KG_15EMPA08',
       'KG_15EMPM05', 'KG_15COCI01', 'KG_15EMPA04', 'KG_15TAJA02',
       'KG_15EMUL01', 'KG_15SELL05', 'KG_15DOSI01', 'KG_15SELL06',
       'KG_15SELL03', 'Hrs_15EMPA09', 'Hrs_15EMPA10', 'Hrs_15PREM01',
       'Hrs_15MEZC08', 'Hrs_15MOLI07', 'Hrs_15AHUM10', 'Hrs_15FORM02',
       'Hrs_15SEPA02', 'Hrs_15MEZC06', 'Hrs_15AHUM01', 'Hrs_15FORM01',
       'Hrs_15SEPA01', 'Hrs_15AHUM09', 'Hrs_15EMBU01', 'Hrs_15EMPM06',
       'Hrs_15TUNE01', 'Hrs_15MEZC07', 'Hrs_15MOLI01', 'Hrs_15MOLI06',
       'Hrs_15EMPA11', 'Hrs_15EMPM01', 'Hrs_15MEZC05', 'Hrs_15MOLI03',
       'Hrs_15HORN01', 'Hrs_15PORC03', 'Hrs_15TUNE03', 'Hrs_15EMPA06',
       'Hrs_15EMPA02', 'Hrs_15MOLI02', 'Hrs_15SELL02', 'Hrs_15MEZC04',
       'Hrs_15MOLI08', 'Hrs_15TUNE02', 'Hrs_15PORC01', 'Hrs_15TUNE04',
       'Hrs_15EMBU03', 'Hrs_15EMPA08', 'Hrs_15EMPM05', 'Hrs_15COCI01',
       'Hrs_15EMPA04', 'Hrs_15TAJA02', 'Hrs_15EMUL01', 'Hrs_15SELL05',
       'Hrs_15DOSI01', 'Hrs_15SELL06', 'Hrs_15SELL03','energia']])

data_numbers_n[['KG_15EMPA09',
       'KG_15EMPA10', 'KG_15PREM01', 'KG_15MEZC08', 'KG_15MOLI07',
       'KG_15AHUM10', 'KG_15FORM02', 'KG_15SEPA02', 'KG_15MEZC06',
       'KG_15AHUM01', 'KG_15FORM01', 'KG_15SEPA01', 'KG_15AHUM09',
       'KG_15EMBU01', 'KG_15EMPM06', 'KG_15TUNE01', 'KG_15MEZC07',
       'KG_15MOLI01', 'KG_15MOLI06', 'KG_15EMPA11', 'KG_15EMPM01',
       'KG_15MEZC05', 'KG_15MOLI03', 'KG_15HORN01', 'KG_15PORC03',
       'KG_15TUNE03', 'KG_15EMPA06', 'KG_15EMPA02', 'KG_15MOLI02',
       'KG_15SELL02', 'KG_15MEZC04', 'KG_15MOLI08', 'KG_15TUNE02',
       'KG_15PORC01', 'KG_15TUNE04', 'KG_15EMBU03', 'KG_15EMPA08',
       'KG_15EMPM05', 'KG_15COCI01', 'KG_15EMPA04', 'KG_15TAJA02',
       'KG_15EMUL01', 'KG_15SELL05', 'KG_15DOSI01', 'KG_15SELL06',
       'KG_15SELL03', 'Hrs_15EMPA09', 'Hrs_15EMPA10', 'Hrs_15PREM01',
       'Hrs_15MEZC08', 'Hrs_15MOLI07', 'Hrs_15AHUM10', 'Hrs_15FORM02',
       'Hrs_15SEPA02', 'Hrs_15MEZC06', 'Hrs_15AHUM01', 'Hrs_15FORM01',
       'Hrs_15SEPA01', 'Hrs_15AHUM09', 'Hrs_15EMBU01', 'Hrs_15EMPM06',
       'Hrs_15TUNE01', 'Hrs_15MEZC07', 'Hrs_15MOLI01', 'Hrs_15MOLI06',
       'Hrs_15EMPA11', 'Hrs_15EMPM01', 'Hrs_15MEZC05', 'Hrs_15MOLI03',
       'Hrs_15HORN01', 'Hrs_15PORC03', 'Hrs_15TUNE03', 'Hrs_15EMPA06',
       'Hrs_15EMPA02', 'Hrs_15MOLI02', 'Hrs_15SELL02', 'Hrs_15MEZC04',
       'Hrs_15MOLI08', 'Hrs_15TUNE02', 'Hrs_15PORC01', 'Hrs_15TUNE04',
       'Hrs_15EMBU03', 'Hrs_15EMPA08', 'Hrs_15EMPM05', 'Hrs_15COCI01',
       'Hrs_15EMPA04', 'Hrs_15TAJA02', 'Hrs_15EMUL01', 'Hrs_15SELL05',
       'Hrs_15DOSI01', 'Hrs_15SELL06', 'Hrs_15SELL03','energia']] = min_max_scaler.transform(data_numbers_n[['KG_15EMPA09',
       'KG_15EMPA10', 'KG_15PREM01', 'KG_15MEZC08', 'KG_15MOLI07',
       'KG_15AHUM10', 'KG_15FORM02', 'KG_15SEPA02', 'KG_15MEZC06',
       'KG_15AHUM01', 'KG_15FORM01', 'KG_15SEPA01', 'KG_15AHUM09',
       'KG_15EMBU01', 'KG_15EMPM06', 'KG_15TUNE01', 'KG_15MEZC07',
       'KG_15MOLI01', 'KG_15MOLI06', 'KG_15EMPA11', 'KG_15EMPM01',
       'KG_15MEZC05', 'KG_15MOLI03', 'KG_15HORN01', 'KG_15PORC03',
       'KG_15TUNE03', 'KG_15EMPA06', 'KG_15EMPA02', 'KG_15MOLI02',
       'KG_15SELL02', 'KG_15MEZC04', 'KG_15MOLI08', 'KG_15TUNE02',
       'KG_15PORC01', 'KG_15TUNE04', 'KG_15EMBU03', 'KG_15EMPA08',
       'KG_15EMPM05', 'KG_15COCI01', 'KG_15EMPA04', 'KG_15TAJA02',
       'KG_15EMUL01', 'KG_15SELL05', 'KG_15DOSI01', 'KG_15SELL06',
       'KG_15SELL03', 'Hrs_15EMPA09', 'Hrs_15EMPA10', 'Hrs_15PREM01',
       'Hrs_15MEZC08', 'Hrs_15MOLI07', 'Hrs_15AHUM10', 'Hrs_15FORM02',
       'Hrs_15SEPA02', 'Hrs_15MEZC06', 'Hrs_15AHUM01', 'Hrs_15FORM01',
       'Hrs_15SEPA01', 'Hrs_15AHUM09', 'Hrs_15EMBU01', 'Hrs_15EMPM06',
       'Hrs_15TUNE01', 'Hrs_15MEZC07', 'Hrs_15MOLI01', 'Hrs_15MOLI06',
       'Hrs_15EMPA11', 'Hrs_15EMPM01', 'Hrs_15MEZC05', 'Hrs_15MOLI03',
       'Hrs_15HORN01', 'Hrs_15PORC03', 'Hrs_15TUNE03', 'Hrs_15EMPA06',
       'Hrs_15EMPA02', 'Hrs_15MOLI02', 'Hrs_15SELL02', 'Hrs_15MEZC04',
       'Hrs_15MOLI08', 'Hrs_15TUNE02', 'Hrs_15PORC01', 'Hrs_15TUNE04',
       'Hrs_15EMBU03', 'Hrs_15EMPA08', 'Hrs_15EMPM05', 'Hrs_15COCI01',
       'Hrs_15EMPA04', 'Hrs_15TAJA02', 'Hrs_15EMUL01', 'Hrs_15SELL05',
       'Hrs_15DOSI01', 'Hrs_15SELL06', 'Hrs_15SELL03','energia']])

data_numbers_n.head()

In [ ]:
start_time = time.time()

sns.set(rc={'figure.figsize':(200,8)}) # Tamaño de la figura
sns.set(style="whitegrid") # Estilo de la figura
sns.boxplot(data = data_numbers_n, linewidth = 3, palette="Set2", fliersize = 5) # Diagrama Box Plot con los datos con escalamiento estándar
sns.despine(left=True)

elapsed_time = time.time() - start_time
print(elapsed_time)

De la anterior figura, se puede afirmar que en la mayoría de los casos si se presentan outliers, estos  sin embargo son muy cerca a la figura del boxplot para cada variables, menos para el horno 01,la porcionadora 03, el tunel 03 y la selladora 03, que presentan demasiados outliers, esto puede ser debido a una inconsistencia en los datos ya que las diferencias entre un dato y otro son muy grandes tanto en Kg, como en horas.

Otros recursos a revisar serían la embutidora 03, la empacadora automatica 08, la empacadora manual 05, el cocinador 01 y la dosificadora 01, los cuales tienen outliers importantes, aproximadamente del 60% de los valores mayores.

# Preparación de los datos

Por el momento, transformaremos algunas variables referentes a las fechas cómo variables categoricas para su posterior analisis

In [ ]:
data["Semana"]= data["Semana"].astype('category')
data["DiaMes"]= data["DiaMes"].astype('category')
data["Mes"]= data["Mes"].astype('category')
data["DiasSem"]= data["DiasSem"].astype('category')

In [ ]:
data

Se convierten las variables categoricas a numericas a traves del metodo dummies

In [ ]:
data_con_var_fechas = pd.get_dummies(data, columns=['Semana', 'DiaMes','Mes','DiasSem'], drop_first=True)

In [ ]:
data_con_var_fechas

Se elimina la caracteristica "Fecha" al ser no relevante para el ejercicio de manera a-priori.

In [ ]:
data_con_var_fechas_2 = data_con_var_fechas.drop("Fecha", axis=1)

In [ ]:
data_con_var_fechas_2

Eliminamos las variables "agua" y energía del conjunto de datos denominado variables predictoras o dependientes "X"

In [ ]:
data_con_var_fechas_sin_y_2=data_con_var_fechas_2.loc[:,data_con_var_fechas_2.columns!='energia']

In [ ]:
data_con_var_fechas_sin_y_2

# Modelación

Realizamos la partición train-test de nuestro conjunto de datos con un tamaño del test del 30%

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data_con_var_fechas_sin_y_2, data_con_var_fechas_2['energia'], \
                                                    test_size=0.3, random_state=0)

A continuación realizaremos la estadarización de la escala de las variables ya que las unidades de las variables son diferentes algunos en Kilogramos y otras en Horas

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
sc = scaler.fit(data_con_var_fechas_sin_y_2)
X_train_n = sc.transform(X_train)
X_test_n = sc.transform(X_test)

Como tecnica de validación cruzada, se utilizará "RepeatedKFold"

In [ ]:
from sklearn.model_selection import RepeatedKFold
rkf = RepeatedKFold(n_splits=10, n_repeats=2, random_state=1)

El primer modelo a analizar será la regresión lineal por minimos cuadrados ordinador

In [ ]:
from sklearn.linear_model import LinearRegression

Como tecnica de busqueda y optimización de hyperparametros de los modelos, se utilizara "GridSearchCV" que permite evaluar y seleccionar de forma sistemática los parámetros de un modelo

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
mod1 = LinearRegression(n_jobs=4)

parameters = {'n_jobs':[3,4]}

mod1_1 = GridSearchCV(estimator=mod1, param_grid = parameters, cv=rkf, scoring='explained_variance',return_train_score=True, n_jobs=-1, verbose=4)
mod1_1.fit(X_train_n, y_train)

Los coeficientes de la regresión lineal son los siguientes:

In [ ]:
mod1_1.best_params_

Utilizamos el modelo de regresión lineal para predecir X_test_n

In [ ]:
mod1_1.best_score_

In [ ]:
results_mod1_1= pd.DataFrame(mod1_1.cv_results_)
results_mod1_1.where(results_mod1_1.params == mod1_1.best_params_).dropna()['mean_train_score'].T

In [ ]:
from sklearn.metrics import explained_variance_score

Yest = mod1_1.predict(X_test_n)

print(f"Explicación de la varianza en X_test_n = {explained_variance_score(y_test,Yest)}")

Según los resultados anteriores, se podría afirmar que el ajuste de la predicción con los valores reales no es la mas adecuada

Ahora se considerara otro modelo el cual es ElasticNet, un tipo de regresión lineal que combina las regularización l1 (forza a que los coeficientes de los predictores tiendan a cero) y l2 (reduce de forma proporcional el valor de todos los coeficientes del modelo pero sin que estos lleguen a cero)

In [ ]:
from sklearn.linear_model import ElasticNet

In [ ]:
mod2 = ElasticNet(random_state=0)

parameters = {'alpha':[0,0.01,0.05,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1], \
              'l1_ratio':[0,0.01,0.05,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]}

mod2_1 = GridSearchCV(estimator=mod2, param_grid = parameters, cv=rkf, scoring='explained_variance',return_train_score=True, n_jobs=-1, verbose=4)
mod2_1.fit(X_train_n, y_train)

Los mejores parametros para el modelo ElasticNet son los siguientes:

In [ ]:
mod2_1.best_params_

{'alpha': 0.4, 'l1_ratio': 0.9}

El mejor resultado para la validación fue:

In [ ]:
mod2_1.best_score_

El mejor resultado de la metrica para la validación fue de 0.8446

Los desempeños en el conjunto de entrenamiento se describen a continuación:

In [ ]:
results_mod2_1= pd.DataFrame(mod2_1.cv_results_)
results_mod2_1.where(results_mod2_1.params == mod2_1.best_params_).dropna()['mean_train_score'].T

El promedio en el score de entrenamiento fue de 0.8638

Ahora se realizará la predicción del conjunto X_test_n y se validara con la metrica explicación de la varianza con respecto al real y_test

In [ ]:
Yest = mod2_1.predict(X_test_n)

print(f"explained_variance_score = {explained_variance_score(y_test,Yest)}")

El resultado fue de 0.8762

In [ ]:
mod2_2 = ElasticNet(alpha=0.4, l1_ratio=0.9,random_state=0)
mod2_3=mod2_2.fit(X_train_n, y_train)
mod2_3.coef_

In [ ]:
d = {'recurso':data_con_var_fechas_sin_y_2.columns, 'coef':mod2_3.coef_}

In [ ]:
df = pd.DataFrame(data=d)

In [ ]:
import plotly.express as px
fig = px.bar(df, x='recurso', y='coef')
fig.show()

Ahora se considerara otro modelo el cual es KNeighborsRegressor

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
from sklearn.metrics import r2_score

start_time = time.time()

mod3 = KNeighborsRegressor(n_jobs=3, algorithm='auto')

parameters = {'n_neighbors':[3,5,7,9,11,13,15], 'weights':['uniform','distance'], 'p':[1,2], \
              'metric':['euclidean','manhattan','chebyshev','minkowski']}

mod3_1 = GridSearchCV(estimator=mod3, param_grid = parameters, cv=rkf, scoring='explained_variance', \
                      return_train_score=True, n_jobs=3,verbose=4)

mod3_1.fit(X_train_n, y_train)

elapsed_time = time.time() - start_time
print(elapsed_time)

In [ ]:
mod3_1.best_params_

In [ ]:
mod3_1.best_score_

In [ ]:
results_mod3_1= pd.DataFrame(mod3_1.cv_results_)
results_mod3_1.where(results_mod3_1.params == mod3_1.best_params_).dropna()['mean_train_score'].T

In [ ]:
Yest = mod3_1.predict(X_test_n)

print(f"explained_variance_score = {explained_variance_score(y_test,Yest)}")

Ahora se considerara otro modelo el cual es DecisionTreeRegressor

In [ ]:
from sklearn.tree import DecisionTreeRegressor

In [ ]:
start_time = time.time()

mod4 = DecisionTreeRegressor(random_state=0)

parameters = {'min_samples_split':[9,10,11,12,13], 
              'criterion':['squared_error','friedman_mse', 'absolute_error', 'poisson'],
             'splitter':['best','random'],
             'max_depth':[4,5,6,7,8,9],
             'min_samples_leaf':[2,3,4,5,6,7],
             'max_features':['auto','sqrt','log2']}

mod4_1 = GridSearchCV(estimator=mod4, param_grid = parameters, cv=rkf, scoring='explained_variance',return_train_score=True, n_jobs=-1,verbose=4)
mod4_1.fit(X_train_n, y_train)

elapsed_time = time.time() - start_time
print(elapsed_time)

In [ ]:
mod4_1.best_params_

In [ ]:
mod4_1.best_score_

In [ ]:
results_mod4_1= pd.DataFrame(mod4_1.cv_results_)
results_mod4_1.where(results_mod4_1.params == mod4_1.best_params_).dropna()['mean_train_score'].T

In [ ]:
Yest = mod4_1.predict(X_test_n)

print(f"explained_variance_score = {explained_variance_score(y_test,Yest)}")

In [ ]:
regr = DecisionTreeRegressor(criterion = 'friedman_mse',
 max_depth = 7,
 max_features = 'auto',
 min_samples_leaf= 5,
 min_samples_split= 11,
 splitter= 'random', random_state=0)

model = regr.fit(X_train_n, y_train)

In [ ]:
from sklearn import tree
fig = plt.figure(figsize=(100,80))
_ = tree.plot_tree(regr, feature_names=data_con_var_fechas_sin_y_2.columns, filled=True)